## PL2 - Phuture Layer 2 Index - Methodology Template

## Configuring packages

In [4]:
import pandas as pd
import decouple
import sys
sys.path.append('../')
from methodology import MethodologyBase
from pycoingecko import CoinGeckoAPI
key =  decouple.config("CG_KEY")
cg = CoinGeckoAPI(api_key=key)

## Index Variables

In [5]:
min_mcap =  0.5e8
min_weight = 0.005
max_weight = 0.3
circ_supply_threshold = 0.05
liveness_threshold = 1 # In days
liquidity_consistency = 90 # In days
coingecko_category = ["layer-2"]

# Calculating max slippage based on liquidity threshold
liq = 2*1e6
liq_os = liq / 2 # one-side liquidity 
trade_value_tiny = 1e2
trade_value = 1e5
max_slippage = (liq_os + trade_value_tiny) / (liq_os + trade_value) - 1
# round to the nearest .01
max_slippage = round(max_slippage, 2)
print(max_slippage)

# Dataframes to remove
stablecoins = pd.DataFrame(cg.get_coins_markets('usd',category='stablecoins')).set_index('id')
ctokens = pd.DataFrame(cg.get_coins_markets('usd',category='compound-tokens')).set_index('id')
atokens = pd.DataFrame(cg.get_coins_markets('usd',category='aave-tokens')).set_index('id')

methodology_class = MethodologyBase(
                                'arbitrum-one',
                                min_mcap,
                                min_weight,
                                max_weight,
                                65535,                                
                                circ_supply_threshold,
                                liveness_threshold,
                                liquidity_consistency,
                                max_slippage,
                                100000,
                                coingecko_category)

-0.09


### Token inclusion criteria

1. **Project and token characteristics**
    1. The project’s token should have been listed on CoinGecko with pricing data spanning at least 6 months prior to the date of inclusion in the index.
    1. The project should have an active community.
    1. No rebasing or deflationary tokens.
    1. Synthetic tokens which derive their value from external price feeds are not permissible. 
    1. The project’s token must not have the ability to pause token transfers.
    1. The project must have a minimum circulating market cap of $50mm.
   <br>
1. **Pricing requirmeents**
    1. Token should have a reliable price feed from one of our authorised providers. 
    <br>
1. **Token Supply requirements**
    1. The project's token must have a circulating supply greater than 20% of the max supply. In cases where a token does not have a max supply, the minting mechanics would need to be assessed.
    <br>
1. **Liquidity Requirements**
    1. The token must be listed on a supported exchange.
    1. The price should experience no more than 12% price impact when executing a $100k trade.
    <br>
1. **Security Requirements**
    1. The project must have been audited by smart contract security professionals with the audit report(s) publicly available. Alternatively, the protocol must have been operating long enough to create a consensus about its safety in the decentralised finance community.
1. **Weighting requirements**
    1. The maximum weight any one token can have is 30%.
    1. All excess weight is proportionally redistributed to all uncapped tokens. After this has been completed, if another token now exceeds the 30% threshold the excess will be redistributed to the remaining uncapped tokens. This process will occur iteratively until there are no tokens that exceed the maximum weight.
    1. Any asset with a weight below 0.5% will be removed from the index.


## Run Methodology

In [6]:

results, liquidity = methodology_class.main(
    df_to_remove=[stablecoins,ctokens,atokens],
    remove_category_assets=['gelato'],
    platforms_to_remove=['ethereum']
    )
liquidity

sora not supported
energi not supported
matic-network on polygon-pos returned an invalid API response
harmony-shard-0 not supported
moonriver not supported
moonbeam not supported
mantle not supported
arbitrum-nova not supported
starknet not supported
metis-andromeda not supported
energi not supported
cartesi on avalanche returned an invalid API response
cartesi on optimistic-ethereum returned an invalid API response
cartesi on arbitrum-one returned an invalid API response
boba not supported


/Users/olivermehr/Documents/GitHub/rebalancing-data/PL2/../methodology.py:444: FutureWarning: The behavior of DataFrame.idxmax with all-NA values, or any-NA and skipna=False, is deprecated. In a future version this will raise ValueError
  slippages["best slippage chain"] = slippages.idxmax(


,arbitrum-one,binance-smart-chain,optimistic-ethereum,polygon-pos,optimal chain,best slippage,best slippage chain
id,,,,,,,
matic-network,NaN,-0.368366,NaN,-0.003972,polygon-pos,-0.003972,polygon-pos
arbitrum,-0.004,NaN,NaN,NaN,arbitrum-one,-0.004000,arbitrum-one
optimism,NaN,NaN,-0.009178,NaN,optimistic-ethereum,-0.009178,optimistic-ethereum
cyberconnect,NaN,-0.989498,-0.423739,NaN,None,-0.423739,optimistic-ethereum
cartesi,NaN,-0.989895,NaN,-0.995339,None,-0.989895,binance-smart-chain
loopring,-1.000,NaN,NaN,NaN,None,-1.000000,arbitrum-one
mantle,NaN,NaN,NaN,NaN,None,NaN,NaN
immutable-x,NaN,NaN,NaN,NaN,None,NaN,NaN
starknet,NaN,NaN,NaN,NaN,None,NaN,NaN


In [7]:
results

,symbol,market_cap,weight,weight_converted,address,blockchain
id,,,,,,
matic-network,MATIC,6.470177e+09,0.333333,21845,0x0000000000000000000000000000000000001010,polygon-pos
arbitrum,ARB,2.861676e+09,0.333333,21845,0x912ce59144191c1204e64559fe8253a0e49e6548,arbitrum-one
optimism,OP,2.462533e+09,0.333333,21845,0x4200000000000000000000000000000000000042,optimistic-ethereum
